In [59]:
import re
from gensim.corpora import Dictionary
from gensim.models import ldamodel
import pandas as pd
import numpy
import random
import string
from nltk.corpus import stopwords

## Data

In [2]:
texts = list(pd.read_csv('./Sberbank Contest/2017/A/train_task1_latest.csv', encoding='utf-8').paragraph.values)

## Colors

In [60]:
csi = '\x1B['
black = csi + '30;1m'
red = csi + '31;1m'    
green = csi + '32;1m'
yellow = csi + '33;1m'
blue = csi + '34;1m'
purple = csi + '35;1m'
cyan = csi + '36;1m'
end = csi + '0m'

list_of_colors = [red, green, yellow, blue, purple, cyan]

## Formatting example

In [62]:
print('Here is a %sred%s word and one in %syellow!%s' % (red, end, yellow, end))
print('Here is a {}red{} word and one in {}yellow!{}'.format(red, end, yellow, end))

Here is a red word and one in yellow!
Here is a red word and one in yellow!


## Coloring words in text

In [51]:
def ColorWordInText(word_list, text, list_of_colors):
    
    text_list = list(text)
    format_list = []
    pos_dict = {}
    
    for word_ix, word in enumerate(word_list):
        text_list = list(text_list)
        for match_pos, match in enumerate(re.finditer(word, ''.join(text_list))):
            text_list.insert(match.start()+2*match_pos,'%')
            text_list.insert(match.end()+2*match_pos+1,'%')
            
            pos_dict[match.start()] = list_of_colors[word_ix]
            
    for col_info in sorted(pos_dict.items(), key=lambda t: t[0]):
        format_list.extend([col_info[1],end])
    
    for match_pos, match in enumerate(re.finditer('%', ''.join(text_list))):
        text_list.insert(match.end()+match_pos,'s')

    #print(''.join(text_list), tuple(format_list))
    print(''.join(text_list) % tuple(format_list))
    #print(''.join(text_list).format(tuple(format_list)))

In [52]:
ColorWordInText(['слово','тестовое'],'тестовое слово слово, тестовое слово тестовое', [red,yellow])

тестовое слово слово, тестовое слово тестовое


In [53]:
print('{} and {}'.format(red, yellow))

 and 


## LDA Highlighting: Framework

In [222]:
def HighlightWordsInText_LDA(list_of_texts, list_of_colors, num_topics=1, top_words=5):
    stop_words = stopwords.words('russian')
    texts = [[x for x in re.sub("[\W+\-\W+]\W+"," ",text).split(' ') if x.lower() not in stop_words ] for text in [list_of_texts]]
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    numpy.random.seed(1)
    model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num_topics, passes=50)

    word_list = []
    colors_list = list_of_colors[:]
    
    for topic in range(num_topics):
        for word_ix in range(top_words):
            word_list.append(model.show_topic(topic, topn=top_words)[word_ix][0])

        if len(colors_list)<top_words*num_topics:
            for c in range(top_words*num_topics-len(colors_list)):
                colors_list.append(black)
    
    return ColorWordInText(word_list, list_of_texts, colors_list)

In [223]:
HighlightWordsInText_LDA(texts[random.randint(0,1000)], list_of_colors, 3, 4)

Женские органы размножения состоят из двух яичников с 4—5 (у примитивных видов — 20-ю) яйцевыми трубочками каждый, парных яйцеводов, объединяющихся в общий яйцевод, открывающийся наружу яйцевыводным отверстием. У Ditrysia, составляющих большинство бабочек, имеется также отдельное копулятивное отверстие, ведущее в копулятивную сумку, проток которой соединяется с общим яйцеводом в месте впадения семяприёмника (сперматеки) и придаточных желез. У Exoporia (Hepialidae) копулятивное и яйцевыводные отверстия также обособлены, но соединены желобком на брюшной поверхности тела. Наконец, у монотризных чешуекрылых копулятивное отверстие соединено с яйцевыводным.


## Highlighting words by topic: Framework

In [320]:
def ColorWordInText(word_list, text, list_of_colors):
    
    text_list = list(text)
    format_list = []
    pos_dict = {}
    
    for word_ix, word in enumerate(word_list):
        text_list = list(text_list)
        for match_pos, match in enumerate(re.finditer(word, ''.join(text_list))):
            text_list.insert(match.start()+2*match_pos,'%')
            text_list.insert(match.end()+2*match_pos+1,'%')
            
            pos_dict[match.start()] = list_of_colors[word_ix]
            
    for col_info in sorted(pos_dict.items(), key=lambda t: t[0]):
        format_list.extend([col_info[1],end])
    
    for match_pos, match in enumerate(re.finditer('%', ''.join(text_list))):
        text_list.insert(match.end()+match_pos,'s')
    
    print(re.sub('%s%s','%s',''.join(text_list)) % tuple(format_list))

In [321]:
def HighlightWordsInText_ByTopic(list_of_texts, list_of_colors, num_topics=1, top_words=5):
    stop_words = stopwords.words('russian')
    texts = [[x for x in re.sub("[\W+\-\W+]\W+"," ",text).split(' ') if x.lower() not in stop_words ] for text in [list_of_texts]]
    dictionary = Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    #numpy.random.seed(1)
    model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=num_topics, passes=50)

    word_list = []
    colors_list = []
    colors_sample = random.sample(list_of_colors,num_topics)
    for topic in range(num_topics):
        for word_ix in range(top_words):
            word_list.append(model.show_topic(topic, topn=top_words)[word_ix][0])
        
        colors_list.extend([colors_sample[topic]]*top_words)

    word_list = list(set(word_list))
    colors_list = colors_list[:len(word_list)]

    return ColorWordInText(word_list, list_of_texts, colors_list)

In [325]:
HighlightWordsInText_ByTopic(texts[random.randint(0,1000)], list_of_colors, 2, 5)

Менее чем через десять лет, в 1986 году, Уимблдонский турнир отметил новый юбилей — сотый розыгрыш чемпионских званий. Как в мужском, так и в женском одиночном разряде в этот год действующие чемпионы успешно отстояли своё звание — молодой Борис Беккер завоевал второй подряд чемпионский титул, а многоопытная Мартина Навратилова пятый подряд и седьмой в общей сложности. Традиционные награды победителям юбилейного чемпионата вручали старейшие живущие чемпионы Уимблдона — соответственно 87-летний Жан Боротра и 90-летняя Кэтлин Годфри. Навратилова, сравнявшись по числу побед в одиночном разряде с довоенной чемпионкой Хелен Уиллз, на этом не оставновилась и к 1990 году довела личный баланс чемпионских титулов до девяти, а через четыре года была близка к тому, чтобы стать десятикратной победительницей в одиночном разряде, проиграв в своём 12-м финале за 22 года испанке Кончите Мартинес[60].


## Theathres reviews

In [9]:
import json
import codecs
import pandas as pd
from nltk.corpus import stopwords # repeated
from functools import reduce
from collections import Counter
import xgboost as xgb

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, GlobalMaxPooling1D
from keras.layers import LSTM
from keras import callbacks
from keras_tqdm import TQDMNotebookCallback
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = stopwords.words('russian')

C:\Users\PDudukin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
def GetFlatList(list_of_lists):
    return reduce(lambda x,y: x+y, list_of_lists)

In [14]:
with codecs.open("reviews.json", encoding = 'utf-8') as json_file:
    json_data = json.load(json_file)

id = 2
print("\x1b[1;30mReview:\x1b[0m",json_data[id]['text'])
print("\x1b[1;30mRating:\x1b[0m",json_data[id]['review_rating'])

Review: 
Очень хороший кинотеатр. Особенно нравится зал номер 4. Везде чисто. И цены хорошие. Один из немногих минусов - это то, что не всегда попадаются чистые 3D-очки. Их бы надо протирать. 

Rating: 4


In [15]:
reviews = []
ratings = []

for id in range(len(json_data)):
    try:
        reviews.append(json_data[id]['text'])
    except KeyError:
        reviews.append('NaN')
    
    try:
        ratings.append(int(json_data[id]['review_rating']))
    except KeyError:
        ratings.append(-1)

In [16]:
data = pd.DataFrame({'raw_review':reviews,'raw_rating':ratings})
data.head(2)

,raw_rating,raw_review
0,4,"\nСколько стоят билеты на комедию "" Кухня: Пос..."
1,1,"\nПочему все мультфильмы в 3D, с малышами неуд..."


In [17]:
data.head(2)

,raw_rating,raw_review
0,4,"\nСколько стоят билеты на комедию "" Кухня: Пос..."
1,1,"\nПочему все мультфильмы в 3D, с малышами неуд..."


In [18]:
test_size = 0.3
for id in range(data.shape[0]):
    if id<int(data.shape[0]*(1-test_size)):
        data.loc[id,'part'] = 'train'
    else:
        data.loc[id,'part'] = 'test'

In [19]:
data.head()

,raw_rating,raw_review,part
0,4,"\nСколько стоят билеты на комедию "" Кухня: Пос...",train
1,1,"\nПочему все мультфильмы в 3D, с малышами неуд...",train
2,4,\nОчень хороший кинотеатр. Особенно нравится з...,train
3,1,"\nочень недоброжелательный, можно сказать враж...",train
4,2,\nпервый раз посетил киноцентр мега синема впе...,train


In [20]:
import re
for id in range(data.shape[0]):
    data.loc[id,'text'] = ' '.join(re.split('\W+',data.raw_review[id].lower())[1:-1])

In [21]:
data.set_value(0,'tokens',0)
data['tokens'] = data['tokens'].astype(object)
for id in range(data.shape[0]):
    
    data.set_value(id,'tokens',[word for word in data.loc[id,'text'].split(' ') if word not in stop_words and word not in ''])

data['rating'] = data['raw_rating'].map(lambda x: 1 if int(x)>3 else -1)
data.head(2)

,raw_rating,raw_review,part,text,tokens,rating
0,4,"\nСколько стоят билеты на комедию "" Кухня: Пос...",train,сколько стоят билеты на комедию кухня последня...,"[сколько, стоят, билеты, комедию, кухня, после...",1
1,1,"\nПочему все мультфильмы в 3D, с малышами неуд...",train,почему все мультфильмы в 3d с малышами неудобн...,"[почему, мультфильмы, 3d, малышами, неудобно, ...",-1


In [22]:
pos = Counter(GetFlatList([x for x in data[data.rating==1]['tokens'].values]))
sorted(pos.items(), key=lambda t: t[1])[-20:-10]

[('место', 437),
 ('просто', 437),
 ('кинотеатра', 445),
 ('зале', 489),
 ('отличный', 565),
 ('фильм', 606),
 ('залы', 621),
 ('билеты', 668),
 ('цены', 725),
 ('удобные', 732)]

In [23]:
neg = Counter(GetFlatList([x for x in data[data.rating==-1]['tokens'].values]))
sorted(neg.items(), key=lambda t: t[1])[-40:-20]

[('15', 322),
 ('места', 322),
 ('20', 323),
 ('т', 324),
 ('смотреть', 325),
 ('экран', 355),
 ('деньги', 357),
 ('билетов', 362),
 ('очки', 365),
 ('сегодня', 367),
 ('5', 367),
 ('3', 399),
 ('туда', 412),
 ('цены', 422),
 ('1', 423),
 ('10', 445),
 ('сеанса', 449),
 ('вообще', 466),
 ('кресла', 466),
 ('билет', 487)]

In [28]:
len(list(set(GetFlatList([x for x in data[data.rating==1]['tokens'].values]))))

24115

In [29]:
len(list(set(GetFlatList([x for x in data[data.rating==-1]['tokens'].values]))))

30122

In [30]:
%time pos_list = GetFlatList([x for x in data[data.rating==+1]['tokens'].values])
%time neg_list = GetFlatList([x for x in data[data.rating==-1]['tokens'].values])
                       
%time pos_words_dict = dict(Counter(pos_list))
%time neg_words_dict = dict(Counter(neg_list))

%time pos_to_delete = list({key: pos_words_dict[key] for key in pos_words_dict.keys() if pos_words_dict[key]<3}.keys())          
%time neg_to_delete = list({key: neg_words_dict[key] for key in neg_words_dict.keys() if neg_words_dict[key]<3}.keys())          

%time clean_pos_list = [x for x in pos_list if x not in pos_to_delete]
%time clean_neg_list = [x for x in neg_list if x not in neg_to_delete]

%time pos_words = [x for x in list(set(clean_pos_list)) if x not in list(set(clean_neg_list))]
%time neg_words = [x for x in list(set(clean_neg_list)) if x not in list(set(clean_pos_list))]

Wall time: 5.63 s
Wall time: 8.16 s
Wall time: 23.1 ms
Wall time: 39.1 ms
Wall time: 9.02 ms
Wall time: 16 ms
Wall time: 1min 29s
Wall time: 2min 51s
Wall time: 1min 9s
Wall time: 1min 9s


In [31]:
print(pos_words[:9])
print(neg_words[:9])

['нужное', 'добродушный', 'отсутсвие', 'значительные', 'попадете', 'мощным', 'заполнены', 'домашней', 'мировой']
['десяти', 'упасть', 'приносит', 'деться', 'непомерно', 'шлагбаум', 'пятница', 'полосы', 'единичный']


In [24]:
tkn = Tokenizer(lower=True)
tkn.fit_on_texts(data.text.values)
data['seq_tokens'] = tkn.texts_to_sequences(data.text.values)
max_text_len = max(data.seq_tokens.apply(len))
fix_text_len = 294
print('Максимальная длина текста:',max_text_len,' слов')
data['seq_tokens'] = list(sequence.pad_sequences(data['seq_tokens'].values, maxlen=fix_text_len))
data.sample(2)

Максимальная длина текста: 294  слов


,raw_rating,raw_review,part,text,tokens,rating,seq_tokens
4483,5,\nНедавно был с семьей в Синема Старе на Дмитр...,train,недавно был с семьей в синема старе на дмитров...,"[недавно, семьей, синема, старе, дмитровском, ...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3244,1,"\nСегодня была на сеансе в зале # 4, в кинотеа...",train,сегодня была на сеансе в зале 4 в кинотеатре ч...,"[сегодня, сеансе, зале, 4, кинотеатре, час, ки...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [33]:
test_x = data[data['part']=='test']['tokens'].apply(list).tolist()
train = data[data['part']=='train']

In [34]:
train_x, valid_x, train_y, valid_y = train_test_split(train['seq_tokens'].apply(list).tolist(),pd.get_dummies(train.rating.values), test_size=0.2)

In [38]:
batch_size = 32
num_features = 42494

model = Sequential()
model.add(Embedding(num_features+1, 50))
model.add(LSTM(100, return_sequences=True)) # new
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='softmax'))

lr = 0.01

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              #optimizer=Adagrad(lr=lr),
              #optimizer=RMSprop(),
              metrics=['accuracy'])

filepath="weights-improvement-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

print('Train...')
early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=0.001, mode='auto')
model.fit(
    train_x,
    train_y.values,
    batch_size=batch_size,
    validation_data=(valid_x, valid_y.values),
    verbose=0,
    epochs=5,
    #callbacks=[TQDMNotebookCallback()]
    callbacks=[checkpoint, TQDMNotebookCallback(), reduce_lr]
)

model.save('TH_Model.h5')

Train...


Epoch 00000: val_loss improved from inf to 0.42240, saving model to weights-improvement-00-0.4224.hdf5


Epoch 00001: val_loss did not improve


Epoch 00002: val_loss did not improve


Epoch 00003: val_loss did not improve


...

Epoch 00005: val_loss did not improve


Epoch 00006: val_loss did not improve


Epoch 00007: val_loss did not improve


Epoch 00008: val_loss did not improve


Epoch 00009: val_loss did not improve


In [39]:
model.load_weights("weights-improvement-01-0.3817.hdf5")
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              #optimizer=Adagrad(lr=lr),
              #optimizer=RMSprop(),
              metrics=['accuracy'])

ValueError: Dimension 1 in both shapes must be equal, but are 50 and 30 for 'Assign_36' (op: 'Assign') with input shapes: [42495,50], [42495,30].

In [40]:
test = data[data['part']=='test']#
pred = model.predict_proba(test['seq_tokens'].apply(list).tolist(),verbose=False)
test['preds'] = list(map(lambda x: 1 if x>=0.5 else -1, pred[:,1]))
test.head(10)

C:\Users\PDudukin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,raw_rating,raw_review,part,text,tokens,rating,seq_tokens,preds
6460,2,"\nДорогие билеты, ограниченный по выбору буфет...",test,дорогие билеты ограниченный по выбору буфет по...,"[дорогие, билеты, ограниченный, выбору, буфет,...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
6461,3,\nСегодня скрепя сердце отправила детей в кино...,test,сегодня скрепя сердце отправила детей в киноте...,"[сегодня, скрепя, сердце, отправила, детей, ки...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
6462,4,"\nДобротный, простой и достаточно дешевый кино...",test,добротный простой и достаточно дешевый кинотеа...,"[добротный, простой, достаточно, дешевый, кино...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
6463,2,"\nПро данный кинотеатр уже много сказано, можн...",test,про данный кинотеатр уже много сказано можно к...,"[данный, кинотеатр, сказано, мириться, душными...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
6464,5,\nХороший кинотеатр.Рядом с метро и рядом с до...,test,хороший кинотеатр рядом с метро и рядом с домо...,"[хороший, кинотеатр, рядом, метро, рядом, домо...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
6465,1,\nНедавно побывал в к/т Родина. Зайдя туда Я с...,test,недавно побывал в к т родина зайдя туда я сраз...,"[недавно, побывал, т, родина, зайдя, туда, сра...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
6466,3,"\nВ кино не была очень давно. но когда ходила,...",test,в кино не была очень давно но когда ходила то ...,"[кино, очень, давно, ходила, частенько, родину...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
6467,4,"\nКинотеатр хороший. Начиная с приемлемых цен,...",test,кинотеатр хороший начиная с приемлемых цен отн...,"[кинотеатр, хороший, начиная, приемлемых, цен,...",1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
6468,1,\nПлюсы - рядом с домом.Минусы:- отвратительна...,test,плюсы рядом с домом минусы отвратительная парк...,"[плюсы, рядом, домом, минусы, отвратительная, ...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1
6469,1,"\nОтвратительное, ужасное, антисанитарное совк...",test,отвратительное ужасное антисанитарное совковое...,"[отвратительное, ужасное, антисанитарное, совк...",-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-1


In [41]:
score, acc = model.evaluate(valid_x, valid_y.values,batch_size=batch_size)

1292/1292 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 5s     


In [42]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.784467310082
Test accuracy: 0.80727554198


In [63]:
def PredictAndColor(review):
    model = load_model('TH_Model.h5')
    model.load_weights("weights-improvement-01-0.3723.hdf5")
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    clean_review = ' '.join(re.split('\W+',review.lower()))
    #print(clean_review)
    tokens = [word for word in clean_review.split(' ') if word not in stop_words and word not in '']
    #print(tokens)
    seq_tokens = tkn.texts_to_sequences([clean_review])
    #print(seq_tokens)
    max_text_len = max(list(map(lambda x: len(x), seq_tokens)))
    seq_tokens = sequence.pad_sequences([seq_tokens[0]], maxlen=max_text_len)
    #print(seq_tokens)
    pred = model.predict(seq_tokens,verbose=False)
    if list(map(lambda x: 1 if x>=0.5 else -1, pred[:,1]))[0] == 1:
        word_list = [x for x in tokens if x in pos_words]
        ColorWordInText(word_list, review, green)
    else:
        word_list = [x for x in tokens if x in neg_words]
        ColorWordInText(word_list, review, red)

In [64]:
test_review = 'непомерно дорогой кинотеатр, никогда больше не приду туда'
PredictAndColor(test_review)

NameError: name 'pos_words' is not defined

### XGBoost Model

In [25]:
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y)

In [26]:
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(4845, 596)
(1615, 596)
(4845,)
(1615,)


In [ ]:
params ={
        'eta':0.01,
        'n_estimators': 793,
        'max_depth': 10,
        'min_child_weight': 1,
        'colsample_bytree': 0.3,
        #'scale_pos_weight': 2.5,
        'objective': 'binary:logistic',
        'eval_metric': 'auc'
        }

do_validation = True

if do_validation:
    dtrain = xgb.DMatrix(X_train.toarray(), label = y_train.tolist())
    dvalid = xgb.DMatrix(X_valid.toarray(), label = y_valid.tolist())
    watchlist = [(dtrain, 'train'), (dvalid, 'train(test)')]
else:
    dtrain = xgb.DMatrix(train_X.toarray(), label = train_y.tolist())
    watchlist = [(dtrain, 'train')]

%time xgb_model = xgb.train(params = params, dtrain = dtrain, evals=watchlist, num_boost_round = params['n_estimators'], verbose_eval=100, early_stopping_rounds=100)

dtest = xgb.DMatrix(test_y.tolist())

data['prediction'] = xgb_model.predict(dtest,ntree_limit=xgb_model.best_iteration)

In [29]:
test_y.values

array([-1, -1,  1, ...,  1,  1,  1], dtype=int64)

In [26]:
set(y_train)

{-1, 1}

In [17]:
train_X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0]], dtype=int64)

In [24]:
train_y.tolist()

[1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -

In [17]:
import numpy as np
data = np.random.rand(5, 10)  # 5 entities, each contains 10 features
label = np.random.randint(2, size=5)  # binary target
dtrain = xgb.DMatrix(data, label=label)

In [18]:
dtrain

In [19]:
print(type(data))

<class 'numpy.ndarray'>


In [21]:
print(type(label))

<class 'numpy.ndarray'>
